
# Reproject ICESat-2 Elevation Points with GeoPandas

In [ ]:
# To surface background logging messages from PROJ
import os
os.environ['USE_PYGEOS'] = '0'
os.environ['PROJ_DEBUG'] = '2' # For showing logs
#os.environ['PROJ_NETWORK'] = 'ON' # Ensure this is 'ON' to get shift grids over the internet
print(os.environ['PROJ_NETWORK'])

In [ ]:
import fiona
import geopandas as gpd

In [ ]:
gpd.show_versions()

In [ ]:
# Saved from sliderule:
#gf = icesat2.atl06p({}, resources=['ATL03_20181019224323_03250112_005_01.h5'])
#gf[:100].to_file('ATL03_20181019224323_03250112_005_01.geojson', driver='GeoJSON')
gf = gpd.read_file('ATL03_20181019224323_03250112_005_01.geojson')

In [ ]:
gf.head(2)

In [ ]:
gf.crs

In [ ]:
# Always check where you data is!
points = gf.reset_index()
points.loc[:, 'time'] = points.time.dt.strftime('%Y-%m-%d')
points.explore(zoom_start=2, column='h_mean')

In [ ]:
# Get bounding box of all of our points
w,s,e,n = gf.union_all().bounds #W, S, E, N
print(w,s,e,n)

In [ ]:
#!projinfo -s EPSG:7912 -t EPSG:9518 -o PROJ --hide-ballpark --spatial-test intersects
!projinfo -s EPSG:7912 -t EPSG:9518 -o PROJ --bbox {w},{s},{e},{n}  --hide-ballpark --spatial-test intersects

In [ ]:
# NOTE: only vshift so only expect vertical coordinate change
points3D = gpd.points_from_xy(gf.geometry.x, gf.geometry.y, gf.h_mean)
gf3D = gpd.GeoDataFrame(geometry=points3D, crs='EPSG:7912')
gf3D.head(2)

In [ ]:
gf3D.crs

In [ ]:
gf3D[:1].get_coordinates(include_z=True)

In [ ]:
gfGeoid = gf3D.to_crs(epsg=9518)
gfGeoid[:1].get_coordinates(include_z=True)

In [ ]:
# SHow debug log

# CAREFUL, geopandas will happily do bogus transforms for you!

import logging
logging.basicConfig(level=logging.DEBUG)

with fiona.Env(CPL_DEBUG=True):
    test = gf3D.to_crs(epsg="2927+5703")

In [ ]:
test.head()